# PLANMAP-Invenio publishing: parsing PLANMAP

In [1]:
PUB_DIR = 'data/publish_data/archive/'
ZIP_DIR = 'data/publish_data/zip/'

In [2]:
import os

# Local api package
import api.invenio

In [3]:
%env INVENIO_API_TOKEN yKsTjnMJe36ut46HdFMo0Z1TAL6JOciYlugtkjllNGqlRlLEZme2zriB4AaG

env: INVENIO_API_TOKEN=yKsTjnMJe36ut46HdFMo0Z1TAL6JOciYlugtkjllNGqlRlLEZme2zriB4AaG


In [4]:
# Invenio server
#
# hostname = 'data.europlanet-gmap.eu'
hostname = '10.72.12.24:5000'
token = os.environ['INVENIO_API_TOKEN']

client = api.invenio.InvenioClient(hostname, token)

In [5]:
import api.planmap
import api.invenio

from importlib import reload
reload(api.invenio)
reload(api.planmap)

import os
import sys
import warnings
import traceback
from glob import glob

warnings.filterwarnings("ignore")


def publish_package(pkg_dir):
    pkg_name = os.path.basename(pkg_dir)
    _,body,mtype,label = pkg_name.split('-')
    body = ({'mer':'mercury', 'mar':'mars', 'moo':'moon'}).get(body.lower())
    # pkg_zip = f"{ZIP_DIR}{body}/{pkg_name}.zip"
    pkg_zip = None

    assert os.path.exists(pkg_dir) and (pkg_zip is None or os.path.exists(pkg_zip))
    print("- Ingesting package:", pkg_name)

    try:
        payload = api.planmap.parse_package(pkg_dir)
        # payload.add_files(pkg_zip)

    except Exception as err:
        print("-- Wrong doing while parsing:", repr(err))
        traceback.print_exc(file=sys.stdout)
        return None

    else:
        if payload is None:
            print("-- Parsing failed because of missing basic information"
                  "Skipping this dataset")
        else:
            print("-- Parsing OK")

    print("-- Publishing it")
    try:
        # Create a draft
        #
        resp = client.create_draft(payload)

        # Publish it
        #
        resp = client.publish_draft(resp['id'])
        _id = resp['id']

    except KeyError as err:
        print("-- Response 'id' not found:", repr(err))
        print(resp)
        traceback.print_exc(file=sys.stdout)
        
    except Exception as err:
        print("-- Wrong doing while publishing:", repr(err))
        traceback.print_exc(file=sys.stdout)

    else:
        print("- Publishing OK", _id)
        return resp


In [9]:
published_list = []

for body in ['mars','mercury','moon']:
    print("Processing packages in", body)

    for pkg_dir in glob(f"{PUB_DIR}/{body}/PM-*"):
        
        resp = publish_package(pkg_dir)
        if resp:
            published_list.append(resp['id'])
            # Just the first one per body, then break loop
            break
        
        print('---')

# resp = publish_package('/Users/chbrandt/Coisas/data/planmap/publish_data/archive/mercury/PM-MER-MS-H05_5cc')

# resp

Processing packages in mars
- Ingesting package: PM-MAR-MS-Crommelin
-- Parsing OK
-- Publishing it
HTTPError('500 Server Error: INTERNAL SERVER ERROR for url: https://10.72.12.24:5000/api/records')
-- Wrong doing while publishing: TypeError("'NoneType' object is not subscriptable")
Traceback (most recent call last):
  File "/var/folders/b1/frq3gywj3ljfqrf1yc7zk06r0000gn/T/ipykernel_22376/3502570098.py", line 51, in publish_package
    resp = client.publish_draft(resp['id'])
TypeError: 'NoneType' object is not subscriptable
---
- Ingesting package: PM-MAR-C-Crommelin
-- Parsing OK
-- Publishing it
HTTPError('500 Server Error: INTERNAL SERVER ERROR for url: https://10.72.12.24:5000/api/records')
-- Wrong doing while publishing: TypeError("'NoneType' object is not subscriptable")
Traceback (most recent call last):
  File "/var/folders/b1/frq3gywj3ljfqrf1yc7zk06r0000gn/T/ipykernel_22376/3502570098.py", line 51, in publish_package
    resp = client.publish_draft(resp['id'])
TypeError: 'Non

KeyboardInterrupt: 

In [7]:
# Open record's page
#
import webbrowser

webbrowser.open(resp['links']['self_html'])
# webbrowser.open(hostname + '/search')

TypeError: 'NoneType' object is not subscriptable